In [ ]:
import pandas as pd
import win32com.client
import os
import re

## ↓↓↓↓ Codigo para encontrar os Index das suas caixas dentro do E-mail ↓↓↓↓

In [ ]:
import win32com.client

outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
root_folder = outlook.Folders.Item(1)  # Acesso à pasta raiz

def traverse_folders(folder, indent=0):
    print(' ' * indent + folder.Name)
    if folder.Folders:
        for i, subfolder in enumerate(folder.Folders):
            print(' ' * (indent + 2) + f"Index: {i}, Name: {subfolder.Name}")
            traverse_folders(subfolder, indent + 4)

traverse_folders(root_folder)



## ↓↓↓↓ Codigo para extração do corpo dos e-mails para um .TXT ↓↓↓↓

In [ ]:


# Conexão com o Outlook
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
root_folder = outlook.Folders.Item(1)  # Acesso à pasta raiz

# Acessando a pasta PELO INDEX
folder = root_folder.Folders[24]

# Criando ou abrindo um arquivo de texto para salvar o código fonte dos e-mails
output_file_path = "output_emails.txt"
with open(output_file_path, "w") as output_file:
    # Itera os e-mails na pasta 
    for message in folder.Items:
        # Obter o corpo de texto do e-mail e remover o cabeçalho
        email_content = message.Body

        # Regexp para encontrar todos os links entre "<>" e ignorar links de imagens
        links = re.findall(r'(?<=<)\b(?!.*\.jpg|.*\.png)https?://\S+?\b(?=>)', email_content)

        # Escrever os links no arquivo de saída
        output_file.write(f"Subject: {message.Subject}\n")
        for link in links:
            output_file.write(link + "\n")
        output_file.write("---------------------------------------------------\n")

print(f"Os links foram salvos em {os.path.abspath(output_file_path)}")

## ↓↓↓↓ Converter o .TXT para uma planilha de excel ↓↓↓↓

In [ ]:
import pandas as pd

# Lendo o arquivo de texto
input_file_path = "output_emails.txt"
output_excel_path = "output_emails.xlsx"

data = []
current_subject = None

with open(input_file_path, "r") as input_file:
    for line in input_file:
        line = line.strip()
        if line.startswith("Subject: "):
            current_subject = line.replace("Subject: ", "")
        elif line.startswith("http"):
            data.append([current_subject, line])

# Convertendo para DataFrame e salvando em um arquivo Excel
df = pd.DataFrame(data, columns=["Subject", "Link"])
df.to_excel(output_excel_path, index=False)

print(f"Os dados foram salvos em {output_excel_path}")
